###基本步驟

In [ ]:
# 連接Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 安裝套件
!pip install line-bot-sdk flask flask-ngrok pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)
# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

In [ ]:
# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("Mi4JsyDs+dGNuZPFlm7SqmlM2YTE0xxDkm8vkRolkSFrA9E2JQQfQKU1hUfayQhk1Po5iOrCCwV3XdwWPxKQH/JwGHxNG4y4qjjc7iLItABk0uTW+KSB/GyHKDoeQyrIE7zzw0jodQpP6Wm3R1BrFgdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("8259df02f27ce24a385057e93d26655f")

In [ ]:
# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body:" + body)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

###Rich Menu 設置

In [ ]:
# 載入linebot.models
from linebot.models import *
import requests

#設定Rich Menu大小、位置及功能
rich_menu = RichMenu(
    size=RichMenuSize(width=2500, height=833),
    selected=False,
    name="Nice richmenu",
    chat_bar_text="Tap here",
    areas =[
        RichMenuArea(
            bounds=RichMenuBounds(x=0, y=0, width=833, height=554),
            action=MessageAction(label="情境互動", text="開啟情境互動任務")),
        RichMenuArea(
            bounds=RichMenuBounds(x=834, y=0, width=833, height=554),
            action=URIAction(label='魚類辨識', uri='https://line.me/R/nv/camera/')),
        RichMenuArea(
            bounds=RichMenuBounds(x=1667, y=0, width=833, height=554),
            action=URIAction(
                label='打開地圖', 
                uri='https://www.nmmst.gov.tw/chhtml/content/312')),
        RichMenuArea(
            bounds=RichMenuBounds(x=0, y=554, width=2500, height=277),
            action=PostbackAction(
                label='其他資訊',
                data="rich_menu_others"
            ))
        ]
)

In [ ]:
# 取得該Rich Menu的id

rich_menu_default_id = line_bot_api.create_rich_menu(rich_menu=rich_menu)
print(rich_menu_default_id)

richmenu-cb3d8bcbea12cdcd31bc1dd63131f24c
richmenu-7b30908a516f64d31ed090db4edd89bf


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive
/content/drive/MyDrive


In [ ]:
# 將特定的Rich Menu(用id來指定要哪個)與想要的圖片綁在一起
import os 
if not os.path.exists('/團體專題Linebot/'):
    os.makedirs('/團體專題Linebot/')

with open("/content/drive/MyDrive/團體專題Linebot/Rich_Menu2_v2.png", 'rb') as f:
    line_bot_api.set_rich_menu_image(rich_menu_default_id, "image/jpeg", f)

In [ ]:
# 將綁好的rich Menu設定為預設圖文選單(不針對使用者id, 所有人都用的)

line_bot_api.set_default_rich_menu(rich_menu_default_id)

###Follow Event-記錄用戶資訊、發送歡迎訊息

In [ ]:
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("/content/drive/MyDrive/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')

    #line_bot_api.link_rich_menu_to_user(event.source.user_id, rich_menu_id)

    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('非常感謝您的關注!')]
    )

###Rich Menu-情境互動連接(暫定Button Template形式進行)

In [ ]:
# 建立Button Template

story_button1 = TemplateSendMessage(
    alt_text="Buttons template",
    template=ButtonsTemplate(
        title="歡迎來到海科館, 接下來請依照提示完成任務, 完成後將能夠獲得神秘小禮物",
        text="準備好要開啟海科館大冒險了嗎",
        thumbnail_image_url="https://imgur.com/a/3vXe3Jh",
        actions=[
            {"type":"message",
            "label":"我準備好了",
            "text":"任務開始!",
            },
            {"type":"message",
            "label":"小等一下",
            "text":"等你喔",
            }
        ]
    )
)

In [ ]:
hint = "\
    a@麵包海星\n\
    b@飯匙鯊\n\
    c@花園鰻小子\n\
    d@獅子魚"

story_dict = {
    "script_a1": "阿尼是一隻小丑魚，鮮豔的橘色搭配白色美麗的條紋看起來帥氣無比。\n不過有件事情一直煩惱著他...就是無論他怎麼努力都交不到女朋友QQ\n聽說今天在智能海洋館裡面有隻美麗的魚也正在尋找自己的伴侶，現在就讓我們來協助阿尼找到真愛吧",
    "script_a2": "一進入館內的第一個區域「潮境之濱」，阿尼發現自己置身美麗的3D投影當中...這時阿尼突然聽到一個低沈的聲音從頭上傳來...\n哇！原來是大鯨魚在說話！",
    "script_d1": "花園鰻小子：\n什麼...想不到你這麼厲害，竟然可以找到我...\n那天我經過海星婆婆的家，聞到一陣陣香味，剛好我的肚子好餓好餓，所以才會偷偷把湯喝掉的，我會親自去和麵包海星婆婆道歉的，真是抱歉！",
    "script_e1": "在阿尼與獅子魚與見面後兩人立刻墜入愛河，從此過著幸福又快樂的生活，阿尼為了感謝您的協助，在此提供一張海科館的八折優惠券作為報答，現在就到販賣區挑一個喜歡的紀念品或是前往互動體驗區展開全新冒險吧！",
    "task_a": "大鯨魚：\n看你帥氣的外表，你就是阿尼對吧...想要找到女朋友的話你可以問問我們館內的媒人婆「麵包海星」婆婆，什麼...你問我她看起來是什麼樣子嗎？\n「麵包海星」一般為5隻腕足，但腕足特別粗短，區分不明顯看起來胖胖的就像是菠蘿麵包一樣喔哈哈哈，她就住在接下來的「潮境之魚」區域當中，我相信他會給你許多有用的資訊的！",
    "task_b": "「麵包海星」婆婆：\n小伙子我正在忙呢！找我有什麼事啊？\n什麼...你想要找女朋友？正好我知道館內有隻美麗的魚也在尋找伴侶呢！\n不過...我昨天剛煮好的濃湯被偷喝了，我現在正忙著找出小偷呢！如果你可以幫忙我的話，我就幫你找到女朋友作為回報！\n現在你有看到旁邊的七公尺超大魚缸嗎，我要你去和「飯匙鯊」警長報案，讓他揪出可惡的小偷！",
    "task_c": "「飯匙鯊」警長：\n是麵包海星婆婆請你來報案的啊...麵包海星婆婆的濃湯是最好喝的！想不到竟然會發生這樣的事...真是可惡！\n依現場殘留的沙子看來，小偷應該是某種會躲藏在沙子裡的生物，而要能夠從門縫進入，想必小偷一定是個苗條的傢伙呢！\n我想小偷應該是走不遠的...也許在接下來的「潮下視界」就可以找他了！",
    "task_d": "麵包海星婆婆：\n做得好阿尼！花園鰻小子已經來和我道歉過並且答應幫我做一個月的家事來補償！\n為了回報你的辛苦我就告訴你吧，在「潮水彩繪」區域中的「獅子魚」小姐正在找尋伴侶！\n獅子魚小姐的外表可是漂亮的喔，美麗的線條斑紋搭配放射狀的胸鰭和背鰭，可是迷倒眾魚呢，不過千萬要小心她可是有毒性的喔！一般人可別離她太近！但我想他會喜歡你這個帥氣又聰明的小帥哥的！\n現在就去找她吧，和她說你是麵包海星婆婆認證的好魚，至於後續就得看你的表現了，好好加油阿尼！",
    "hint": f"請依據對話中的線索，拍攝正確物種之照片並傳送於對話筐內\n(若是沒有找到正確魚類的可以再次上傳照片嘗試喔)\n\n#現行先用<numstQuestion>@<species>來代替圖片\nex:\n{hint}",
    "麵包海星": "麵包海星婆婆有「5隻特別粗短的腕足，看起來胖胖像是菠蘿麵包」請再找找～",
    "飯匙鯊": "飯匙鯊頭部前圓而寬闊，頭與胸鰭非常容易區別，他就在七尺大魚缸中游泳呢！請再找找～",
    "花園鰻小子": "根據線索花園鰻小子小偷是個苗條的傢伙，常常會躲在沙子裡面，而且跟線報得知他的身上有著明顯的黑色斑點，請再找找吧～",
    "獅子魚": "這不是獅子魚小姐啦～\n獅子魚擁有美麗的線條斑紋搭配放射狀的胸鰭和背鰭，請再找找看吧",
    "a": "麵包海星",
    "b": "飯匙鯊",
    "c": "花園鰻小子",
    "d": "獅子魚",
}

In [ ]:
'''

準備QuickReply的Button


'''

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)

# 創建QuickReplyButton 

## 點擊後，以用戶身份發送文字消息
## MessageAction
story_quick_reply_start = QuickReplyButton(
    action=PostbackAction(
        label="故事開始",
        display_text="故事開始",
        data="start"
    )
)
story_quick_reply_script_a1 = QuickReplyButton(
    action=PostbackAction(
        label="進入「潮境之濱」",
        display_text="進入「潮境之濱」",
        data="script_a1"    
    )
)
story_quick_reply_script_a2 = QuickReplyButton(
    action=PostbackAction(
        label="與大鯨魚對話",
        display_text="與大鯨魚對話", 
        data="script_a2"
    )
)
story_quick_reply_script_d1 = QuickReplyButton(
    action=PostbackAction(
        label="再訪麵包海星婆婆",
        display_text="再訪麵包海星婆婆", 
        data="script_d1"
    )
)
story_quick_reply_script_e1 = QuickReplyButton(
    action=PostbackAction(
        label="終於...見到「獅子魚」小姐",
        display_text="終於...見到「獅子魚」小姐", 
        data="script_e1"
    )
)
story_quick_reply_task_a = QuickReplyButton(
    action=PostbackAction(
        label="進入「潮境之魚」，尋找「麵包海星」婆婆",
        display_text="進入「潮境之魚」，尋找「麵包海星」婆婆",
        data="hint"
    )
)
story_quick_reply_task_b = QuickReplyButton(
    action=PostbackAction(
        label="尋找「飯匙鯊」警長",
        display_text="尋找「飯匙鯊」警長",
        data="hint"
    )
)
story_quick_reply_task_c = QuickReplyButton(
    action=PostbackAction(
        label="進入「潮下視界」，尋找小偷", 
        display_text="進入「潮下視界」，尋找小偷",
        data="hint"
    )
)
story_quick_reply_task_d = QuickReplyButton(
    action=PostbackAction(
        label="進入「潮水彩繪」」，尋找「獅子魚」小姐", 
        display_text="進入「潮水彩繪」」，尋找「獅子魚」小姐",
        data="hint"
    )
)

# 相機 & 相簿
cameraQuickReplyButton = QuickReplyButton(
    action=URIAction(
        label="打開相機",
        uri="https://line.me/R/nv/camera/"
    )
)
cameraRollQRB = QuickReplyButton(
    action=CameraRollAction(label="選擇照片")
)

In [ ]:
# 建立觸發回覆

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if event.message.text == "開啟情境互動任務":
        line_bot_api.reply_message(event.reply_token, 
                                story_button1)
    if event.message.text == "任務開始!":
        flex_message = TextSendMessage(
            text="請點擊下方泡泡提示進行故事",
            quick_reply=QuickReply(
                items=
                [
                    story_quick_reply_start
                ]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message
            ]
        )
    if event.message.text[1] == "@":
        info = event.message.text.split("@")
        if info[0] == "a" and info[1] == "麵包海星":
            flex_message = TextSendMessage(
            text=story_dict["task_b"],
            quick_reply=QuickReply(
                items=[
                story_quick_reply_task_b
                ]
            )
            )
            line_bot_api.reply_message(
                event.reply_token,
                [
                    flex_message
                ]
            )
        elif info[0] == "b" and info[1] == "飯匙鯊":
            flex_message = TextSendMessage(
            text=story_dict["task_c"],
            quick_reply=QuickReply(
                items=[
                story_quick_reply_task_c
                ]
            )
            )
            line_bot_api.reply_message(
                event.reply_token,
                [
                    flex_message
                ]
            )
        elif info[0] == "c" and info[1] == "花園鰻小子":
            flex_message = TextSendMessage(
            text=story_dict["script_d1"],
            quick_reply=QuickReply(
                items=[
                story_quick_reply_script_d1
                ]
            )
            )
            line_bot_api.reply_message(
                event.reply_token,
                [
                    flex_message
                ]
            )
        elif info[0] == "d" and info[1] == "獅子魚":
            flex_message = TextSendMessage(
            text="獅子魚小姐: 你找到我啦～",
            quick_reply=QuickReply(
                items=[
                story_quick_reply_script_e1
                ]
            )
            )
            line_bot_api.reply_message(
                event.reply_token,
                [
                    flex_message
                ]
            )
        else:
            line_bot_api.reply_message(
                event.reply_token,
                [TextSendMessage(f"這不是{story_dict[info[0]]}喔，" + story_dict[story_dict[info[0]]])]
            )
            

In [ ]:
# '''

# 以QuickReply封裝該些QuickReply Button

# '''
# ## 設計QuickReplyButton的List
# quickReplyList = QuickReply(
#     items = [textQuickReplyButton, cameraQuickReplyButton, cameraRollQRB, postbackQRB]
# )

In [ ]:
'''

製作TextSendMessage，並將剛封裝的QuickReply放入

'''
## 將quickReplyList 塞入TextSendMessage 中 
# from linebot.models import (
#     TextSendMessage,
# )
# quick_reply_text_send_message = TextSendMessage(text='發送問題給用戶，請用戶回答', quick_reply=quickReplyList)

'\n\n製作TextSendMessage，並將剛封裝的QuickReply放入\n\n'

'\n\n製作TextSendMessage，並將剛封裝的QuickReply放入\n\n'

###Rich Menu-其他功能Quickreply Button建立

In [ ]:
@handler.add(PostbackEvent)
def handle_postback_message(event):
    # user_profile = line_bot_api.get_profile(event.source.user_id)
    if event.postback.data == 'rich_menu_others':
        flex_message = TextSendMessage(
            text='最新活動及魚種介紹',
            quick_reply=QuickReply(
                items=[
                QuickReplyButton(action=MessageAction(label="其他活動", text="其他活動")),
                QuickReplyButton(action=MessageAction(label="魚種介紹", text="魚種介紹")),
                QuickReplyButton(action=URIAction(
                    label='進入官網', 
                    uri='https://www.nmmst.gov.tw/chhtml/'
                ))]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message
            ]
        )
    elif event.postback.data == "start":
        flex_message = TextSendMessage(
            text=story_dict["script_a1"],
            quick_reply=QuickReply(
                items=[
                story_quick_reply_script_a1
                ]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message
            ]
        )
    elif event.postback.data == "script_a1":
        flex_message = TextSendMessage(
            text=story_dict["script_a2"],
            quick_reply=QuickReply(
                items=[
                story_quick_reply_script_a2
                ]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message
            ]
        )
    elif event.postback.data == "script_a2":
        flex_message = TextSendMessage(
            text=story_dict["task_a"],
            quick_reply=QuickReply(
                items=[
                story_quick_reply_task_a
                ]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message
            ]
        )
    elif event.postback.data == "script_d1":
        flex_message = TextSendMessage(
            text=story_dict["task_d"],
            quick_reply=QuickReply(
                items=[
                story_quick_reply_task_d
                ]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message
            ]
        )
    elif event.postback.data == "script_e1":
        line_bot_api.reply_message(
            event.reply_token,
            [
                TextSendMessage(story_dict["script_e1"])
            ]
        )
    elif event.postback.data == "hint":
        flex_message = TextSendMessage(
            text=story_dict["hint"],
            quick_reply=QuickReply(
                items=[
                cameraQuickReplyButton,
                cameraRollQRB
                ]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message
            ]
        )

In [ ]:
app.run()